In [1]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
!unzip bank+marketing.zip
!unzip bank.zip

--2025-10-04 08:27:17--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip.1’

bank+marketing.zip.     [  <=>               ] 999.85K  2.65MB/s    in 0.4s    

2025-10-04 08:27:17 (2.65 MB/s) - ‘bank+marketing.zip.1’ saved [1023843]

Archive:  bank+marketing.zip
replace bank.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank-additional.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  bank.zip
replace bank-full.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank-names.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace bank.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [60]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold

In [61]:
df = pd.read_csv('bank-full.csv', sep = ';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [62]:
df = df[['age',
'job',
'marital',
'education',
'balance',
'housing',
'contact',
'day',
'month',
'duration',
'campaign',
'pdays',
'previous',
'poutcome',
'y']]

In [63]:
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [64]:
df['y'] = (df['y']=='yes').astype(int)

In [65]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [66]:
df_train, df_val= train_test_split(df_full_train, test_size = 0.25, random_state = 1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_val['y']
del df_test['y']

In [67]:
for col in ['age','balance','day','duration','campaign','pdays','previous']:
  auc = roc_auc_score(y_train, df_train[col])
  if auc < 0.5:
    auc = 1-auc
  print(col, auc)

age 0.5121857175273441
balance 0.5888313805382317
day 0.525957882383908
duration 0.8147002759670778
campaign 0.5714543015682159
pdays 0.5901276247352144
previous 0.5985653242764153


In [68]:
dv = DictVectorizer(sparse=False)
X_train_2=dv.fit_transform(df_train.to_dict(orient = 'records'))
X_val_2=dv.transform(df_val.to_dict(orient = 'records'))

In [69]:
model = LogisticRegression(solver='liblinear', C= 1.0, max_iter = 1000, random_state = 1)
model.fit(X_train_2, y_train)

LogisticRegression(max_iter=1000, random_state=1, solver='liblinear')

In [70]:
y_pred = model.predict_proba(X_val_2)[:, 1]
auc = roc_auc_score(y_val, y_pred)
round(auc,3)

np.float64(0.9)

In [71]:
thresholds = np.linspace(0, 1.0, 101)
for threshold in thresholds:
  y_pred_binary = (y_pred >= threshold).astype(int)
  tp = ((y_pred_binary == 1) & (y_val == 1)).sum()
  tn = ((y_pred_binary == 0) & (y_val == 0)).sum()
  fp = ((y_pred_binary == 1) & (y_val == 0)).sum()
  fn = ((y_pred_binary == 0) & (y_val == 1)).sum()
  precision = tp/(tp+fp) if (tp + fp) > 0 else 0.0
  recall = tp/(tp+fn)
  print(threshold, precision, recall)

0.0 0.12143331121433311 1.0
0.01 0.13509508520622376 0.9963570127504554
0.02 0.15882011043301367 0.9954462659380692
0.03 0.1863854595336077 0.9899817850637522
0.04 0.21939194741166804 0.9726775956284153
0.05 0.2541222114451988 0.9544626593806922
0.06 0.28903372060073673 0.9289617486338798
0.07 0.31694861155442067 0.9043715846994536
0.08 0.3425238600212089 0.8825136612021858
0.09 0.3665632273079907 0.860655737704918
0.1 0.38280281107895825 0.843351548269581
0.11 0.39884649511978704 0.8187613843351548
0.12 0.41442307692307695 0.785063752276867
0.13 0.42827868852459017 0.761384335154827
0.14 0.44036697247706424 0.7431693989071039
0.15 0.4503386004514673 0.726775956284153
0.16 0.46626865671641793 0.7112932604735883
0.17 0.4756857855361596 0.6948998178506375
0.18 0.4878688524590164 0.6775956284153005
0.19 0.49897048730267674 0.6621129326047359
0.2 0.5067615658362989 0.6484517304189436
0.21 0.5155555555555555 0.6338797814207651
0.22 0.5289128758673862 0.6247723132969034
0.23 0.53790322580645

In [72]:
results_f1 = {}
for threshold in thresholds:
  y_pred_binary = (y_pred >= threshold).astype(int)
  tp = ((y_pred_binary == 1) & (y_val == 1)).sum()
  tn = ((y_pred_binary == 0) & (y_val == 0)).sum()
  fp = ((y_pred_binary == 1) & (y_val == 0)).sum()
  fn = ((y_pred_binary == 0) & (y_val == 1)).sum()
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  F1 = 2 * precision * recall / (precision + recall)
  results_f1[threshold] = F1

/tmp/ipython-input-727493379.py:8: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)


In [73]:
max(results_f1, key =results_f1.get)

np.float64(0.22)

In [92]:
cv = KFold(n_splits=5, shuffle=True, random_state=1)
model = LogisticRegression(solver='liblinear', C= 1.0, max_iter = 1000, random_state = 1)


In [96]:
def train(df_train, y_train, model):
    dicts = df_train.drop(['y'],axis= 1).to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model.fit(X_train, y_train)

    return dv, model

In [97]:
def predict(df, dv, model):
    dicts = df.drop(['y'],axis = 1).to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [98]:
from tqdm.auto import tqdm

In [99]:
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    y_train = df_train.y.values
    y_val = df_val.y.values

    dv, model = train(df_train, y_train, model)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

    print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=0.001 0.899 +- 0.000
C=0.001 0.899 +- 0.000
C=0.001 0.903 +- 0.006
C=0.001 0.904 +- 0.005
C=0.001 0.906 +- 0.006


In [101]:
n_splits = 5

scores = []
for c in [0.000001, 0.001, 1]:
  kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
  model = LogisticRegression(solver='liblinear', C= c, max_iter = 1000, random_state = 1)
  for train_idx, val_idx in kfold.split(df_full_train):
      df_train = df_full_train.iloc[train_idx]
      df_val = df_full_train.iloc[val_idx]
      y_train = df_train.y.values
      y_val = df_val.y.values

      dv, model = train(df_train, y_train, model)
      y_pred = predict(df_val, dv, model)

      auc = roc_auc_score(y_val, y_pred)
      scores.append(auc)

      print('C=%s %.3f +- %.3f' % (c, np.mean(scores), np.std(scores)))

C=1e-06 0.685 +- 0.000
C=1e-06 0.692 +- 0.008
C=1e-06 0.699 +- 0.011
C=1e-06 0.701 +- 0.011
C=1e-06 0.701 +- 0.009
C=0.001 0.726 +- 0.055
C=0.001 0.745 +- 0.069
C=0.001 0.760 +- 0.076
C=0.001 0.772 +- 0.079
C=0.001 0.781 +- 0.080
C=1 0.792 +- 0.084
C=1 0.801 +- 0.085
C=1 0.809 +- 0.087
C=1 0.816 +- 0.088
C=1 0.823 +- 0.088
